# Testing the scaling limits of GTSAM on 1dsfm dataset

![image](media/1dsfmRef.png)

The dataset can be found here: https://www.cs.cornell.edu/projects/1dsfm/

## Pre-Processing 

### Need for preprocessing

"Removed the incldued gt_bundle.out files, since there are indexing errors with these files that make them unusable (confirmed by the author)." - Nikolaus Demmel (https://gitlab.vision.in.tum.de/rootba/rootba_data/-/tree/05cc69b78da5792ced0b0e7cbd6fd9ec46cd9763/1dsfm)

This also allows for a real-world example with minimal preprocessing and existence of high noise and outliers in the data.

###  Preprocessing Pipeline

Before performing Bundle Adjustment (BA) scaling experiments in GTSAM, the raw, unordered image collections from the **1DSfM dataset** must be transformed into a structured factor graph. We utilize **COLMAP**, a state-of-the-art Structure-from-Motion (SfM) pipeline, as the frontend to achieve this. 

### Why is this step necessary?

GTSAM is a backend optimization library; it requires a defined **NonlinearFactorGraph** and good **Initial Values** to converge. COLMAP solves three critical problems:

1.  **Data Association (Feature Matching):** We must determine which 2D pixel $(u,v)$ in Image $A$ corresponds to the same 3D landmark as pixel $(u',v')$ in Image $B$. COLMAP uses SIFT descriptors and geometric verification to generate these feature tracks.
2.  **Initialization (Sparse Reconstruction):** Bundle Adjustment is a non-convex optimization problem. If initialized with identity poses and zero-point coordinates, the optimizer will get stuck in local minima. COLMAP provides a robust initial estimate for Camera Poses $(R, t)$ and 3D Points $(X, Y, Z)$.
3.  **Graph Topology:** The reconstruction determines the sparsity pattern of the graph—defining exactly which cameras observe which landmarks.

---

### The Execution Pipeline

The preprocessing was executed via a shell script using a Dockerized version of COLMAP (for GPU acceleration). The pipeline consists of four sequential stages:

```bash
# 1. Feature Extraction
# Detects SIFT features. 'single_camera 0' allows for varying image dimensions.
colmap feature_extractor \
    --database_path database.db \
    --image_path /images \
    --ImageReader.single_camera 0

# 2. Feature Matching
# Associates features across images. We utilize a Vocabulary Tree (FAISS) 
# for O(N) efficiency on large datasets (Montreal/Piccadilly).
colmap vocab_tree_matcher \
    --database_path database.db \
    --VocabTreeMatching.vocab_tree_path vocab_tree_faiss.bin

# 3. Sparse Reconstruction (The SfM Frontend)
# Incrementally registers images to build the initial 3D model.
colmap mapper \
    --database_path database.db \
    --image_path /images \
    --output_path sparse

# 4. Conversion to Bundler Format
# Exports the binary COLMAP model to a text-based format parseable by our scripts.
colmap model_converter \
    --input_path sparse/0 \
    --output_path colmap_bundle.out \
    --output_type Bundler



The COLMAP pipeline was run with sparse reconstruction for a decent initial guess and due to time needed for better reconstruction (multiple hours). 

### Output Artifacts

The pipeline generates two critical files that serve as the interface between the COLMAP frontend and the GTSAM backend:


| File Name | Content Description | Role in GTSAM |
| :--- | :--- | :--- |
| **`colmap_bundle.out.bundle.out`** | **The Geometry File** (Bundler Format)<br>• **Header:** Camera count & Point count.<br>• **Cameras:** Focal length ($f$), Distortion ($k_1, k_2$), Rotation ($R$), Translation ($t$).<br>• **Structure:** 3D landmarks $(X, Y, Z)$, colors, and the list of 2D feature observations per point. | **Core Input:** Parsed to create the initial `gtsam.Values` (Pose3, Point3) and the `gtsam.NonlinearFactorGraph`. |
| **`colmap_bundle.out.list.txt`** | **The Association Index**<br>• A plain text list mapping the internal Bundle index (e.g., Camera 0) to the original image filename (e.g., `image_0005.jpg`). | **Metadata:** Used to verify the ordering of cameras and associate optimized poses back to the source images for visualization. |




## GTSAM test bench

In [1]:
import numpy as np
import pandas as pd
import gc
import gtsam
import pickle
import os
from helper_1dsfm import (
    load_1dsfm_data,
    build_factor_graph,
    run_bundle_adjustment,
    run_safe_bundle_adjustment,
    print_results,
    save_results,
    load_results,
    plot_reconstruction,
    C, P  # Symbol shortcuts
)

In [2]:
RESULTS_FILE = '1dsfm_results.pkl'
RESULTS_CSV = '1dsfm_scaling_results.csv'

# Dataset paths - update with your actual 1DSfM file paths
# Each entry: (name, bundle_file, list_file)
DATASET_PATHS = [
    ('alamo', 
     '/home/anatharv1/GTSAM-Project/BA_datasets/1dsfm/Alamo/colmap_bundle.out.bundle.out', 
     '/home/anatharv1/GTSAM-Project/BA_datasets/1dsfm/Alamo/colmap_bundle.out.list.txt'),
    
    ('montreal_notre_dame', 
     '/home/anatharv1/GTSAM-Project/BA_datasets/1dsfm/Montreal_Notre_Dame/colmap_bundle.out.bundle.out', 
     '/home/anatharv1/GTSAM-Project/BA_datasets/1dsfm/Montreal_Notre_Dame/colmap_bundle.out.list.txt'),

    ('roman_forum', 
     '/home/anatharv1/GTSAM-Project/BA_datasets/1dsfm/Roman_Forum/colmap_bundle.out.bundle.out', 
     '/home/anatharv1/GTSAM-Project/BA_datasets/1dsfm/Roman_Forum/colmap_bundle.out.list.txt'),
    
    ('picaddilly', 
     '/home/anatharv1/GTSAM-Project/BA_datasets/1dsfm/Piccadilly/colmap_bundle.out.bundle.out', 
     '/home/anatharv1/GTSAM-Project/BA_datasets/1dsfm/Piccadilly/colmap_bundle.out.list.txt'),
    
    ('trafalgar', 
     '/home/anatharv1/GTSAM-Project/BA_datasets/1dsfm/Trafalgar/colmap_bundle.out.bundle.out', 
     '/home/anatharv1/GTSAM-Project/BA_datasets/1dsfm/Trafalgar/colmap_bundle.out.list.txt'),
    
]

In [ ]:

import time
import os

def calculate_bundler_sparsity(file_path, dataset_name):
    print(f"[{dataset_name}] Parsing {os.path.basename(file_path)}...")
    start_time = time.time()
    
    try:
        with open(file_path, 'r') as f:
            # 1. Read Header
            # Skip comment lines if any (Bundler files often start with # Bundle file v0.3)
            header_line = f.readline()
            while header_line.startswith("#"):
                header_line = f.readline()
                
            header = header_line.split()
            num_cameras = int(header[0])
            num_points = int(header[1])
            
            print(f"  > Header: {num_cameras} cameras, {num_points} points")

            # 2. Skip Camera Parameters
            # Each camera has 5 lines of parameters (f/k, R, t)
            # We strictly assume standard Bundler format here
            for _ in range(num_cameras * 5):
                f.readline()

            # 3. Parse Point Observations
            # Points are listed one by one. Each point has 3 lines:
            # Line 1: Position (X Y Z)
            # Line 2: Color (R G B)
            # Line 3: View list (count cam_idx key_idx x y ...)
            
            rows = [] # Camera indices
            cols = [] # Point indices
            
            # Using a loop is necessary because the data is unstructured (variable length lines)
            for pt_idx in range(num_points):
                # Skip Position and Color lines
                f.readline() 
                f.readline()
                
                # Read View List
                view_line = f.readline().split()
                count = int(view_line[0])
                
                if count > 0:
                    # View list format: <count> [cam_idx key_idx x y] [cam_idx key_idx x y] ...
                    # We extract every 4th element starting from index 1 to get camera indices
                    # slice: view_line[1::4] gives all camera indices for this point
                    cam_indices = [int(x) for x in view_line[1::4]]
                    
                    rows.extend(cam_indices)
                    cols.extend([pt_idx] * count)

            # 4. Build Sparse Visibility Matrix (V)
            # Rows = Cameras, Cols = Points
            data = np.ones(len(rows), dtype=bool)
            V = sp.csr_matrix((data, (rows, cols)), shape=(num_cameras, num_points))
            
            # 5. Compute Schur Pattern (S = V * V.T)
            # This is the heavy lifting: Matrix multiplication
            print(f"  > Computing V @ V.T ({V.nnz} observations)...")
            S_pattern = V @ V.T
            
            # 6. Calculate Sparsity Metrics
            non_zeros = S_pattern.nnz
            total_elements = num_cameras ** 2
            density = non_zeros / total_elements
            sparsity = 1.0 - density
            
            elapsed = time.time() - start_time
            print(f"  > Done in {elapsed:.2f}s")
            
            return {
                "name": dataset_name,
                "cameras": num_cameras,
                "points": num_points,
                "density": density,
                "sparsity": sparsity,
                "nnz": non_zeros,
                "error": None
            }

    except Exception as e:
        return {"name": dataset_name, "error": str(e)}


for name, bundle_path, list_path in DATASET_PATHS:
    if not os.path.exists(bundle_path):
        print(f"{name:<20} | {'FILE NOT FOUND':<30}")
        continue
        
    stats = calculate_bundler_sparsity(bundle_path, name)
    
            
    print(f"{name:<20} | {stats['cameras']:<6} | {stats['sparsity']*100:>9.2f}% ")

### About the dataset



| File Name | Num Images| Num Points | Num Observations | Num Nodes | Num Factors | Schur's complement matrix sparsity |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- |
| 'Alamo'| 877 | 99351 | 1098094 |  100228 | 1098094 | 36.30% |
| 'Montreal Notre Dame'| 571 | 93897 | 879864 | 946468 | 879864 | 37.70% |
| 'Roman Forum'| 1490 | 182050 | 1455631 | 183540 | 1455631 | 74.07% |
| 'Piccadilly'| 2994 | 229448 | 2024968 | 232442 | 2024968 | 63.84% |
| 'Trafalgar'| NA | NA | NA | NA | NA | NA |

- The Trafalgar bundle.out file was corrupted and shows garbage data, the reason is poor results from COLMAP pipeline

In [3]:
# Load existing results
all_results = load_results(RESULTS_FILE)
if all_results:
    print(f"Loaded {len(all_results)} existing results from {RESULTS_FILE}")
    print(f"Completed datasets: {[r['name'] for r in all_results]}")
else:
    print("No existing results found. Starting fresh.")
    all_results = []

No existing results found. Starting fresh.


In [ ]:
def save_results_csv(results, filename):
    """Save results to CSV file."""
    if not results:
        return
    
    # Create a simplified version for CSV (no lists)
    csv_data = []
    for r in results:
        csv_entry = {k: v for k, v in r.items() 
                    if not isinstance(v, (list, np.ndarray))}
        csv_data.append(csv_entry)
    
    df = pd.DataFrame(csv_data)
    df.to_csv(filename, index=False)
    print(f"  CSV saved to {filename}")

# Helpers for memory monitoring & management
def check_memory_status():
    """Print current memory status."""
    import psutil
    mem = psutil.virtual_memory()
    print(f"\nMemory Status:")
    print(f"  Total: {mem.total / (1024**3):.1f} GB")
    print(f"  Available: {mem.available / (1024**3):.1f} GB")
    print(f"  Used: {mem.used / (1024**3):.1f} GB ({mem.percent}%)")
    print(f"  Free: {mem.free / (1024**3):.1f} GB")


def force_cleanup():
    """Force garbage collection and print memory freed."""
    import psutil
    
    before = psutil.virtual_memory().used / (1024**3)
    gc.collect()
    after = psutil.virtual_memory().used / (1024**3)
    freed = before - after
    
    print(f"\nGarbage collection completed")
    if freed > 0.01:
        print(f"  Freed: {freed:.2f} GB")
    check_memory_status()



In [ ]:
def run_single_dataset(idx, verbose=True, plot=True, max_points=10000,
                        lambda_initial=1e-3, lambda_upper_bound=1e12,
                        use_robust=True, use_bundler_camera=False):
    """
    Run bundle adjustment on a single 1DSfM dataset with aggressive memory management.
    
    Args:
        idx: dataset index
        verbose: print detailed output
        plot: if True, generate visualization before cleanup
        max_points: maximum number of points to plot
        lambda_initial: initial damping parameter (default: 1e-3, conservative for COLMAP)
        lambda_upper_bound: maximum lambda value (default: 1e12)
        use_robust: if True, use Huber robust loss for outliers
        use_bundler_camera: if True, use Cal3Bundler (False recommended for COLMAP exports)
    """
    global all_results
    
    name, bundle_file, list_file = DATASET_PATHS[idx]
    
    # Check if already completed
    completed_names = [r['name'] for r in all_results]
    if name in completed_names:
        print(f"\n{'='*70}")
        print(f"DATASET {idx+1}: {name} - ALREADY COMPLETED, SKIPPING")
        print(f"{'='*70}")
        return True
    
    # Force garbage collection before starting
    gc.collect()
    
    print(f"\n{'='*70}")
    print(f"DATASET {idx+1}: {name}")
    print(f"{'='*70}")
    
    try:
        # Load data
        cameras, points, observations, image_names = load_1dsfm_data(bundle_file, list_file)
        
        # Build graph with Bundler camera model (COLMAP convention)
        graph, initial = build_factor_graph(
            cameras, points, observations,
            use_robust=use_robust,
            filter_behind_camera=False,  # Don't filter for COLMAP data
            use_bundler_camera=use_bundler_camera
        )
        
        # Run optimization with custom lambda settings
        result, metrics = run_bundle_adjustment(
            graph, initial, 
            max_iterations=50,
            verbose=verbose,
            lambda_initial=lambda_initial,
            lambda_upper_bound=lambda_upper_bound
        )
        
        # Print results
        print_results(name, cameras, points, observations, metrics)
        
        # Store results
        result_entry = {
            'name': name,
            'num_cameras': len(cameras),
            'num_points': len(points),
            'num_observations': len(observations),
            **metrics
        }
        all_results.append(result_entry)
        
        # Save immediately
        save_results(all_results, RESULTS_FILE)
        save_results_csv(all_results, RESULTS_CSV)
        
        # Plot BEFORE cleanup (so we don't re-optimize)
        if plot:
            print("\n" + "="*70)
            print("GENERATING VISUALIZATION")
            print("="*70)
            try:
                plot_reconstruction(
                    cameras, points,
                    initial, 
                    result, 
                    name,
                    max_points=max_points,
                    save_path=f"{name}_reconstruction.html"
                )
                print(f"✓ Plot saved to: {name}_reconstruction.html")
            except Exception as e:
                print(f"  Plotting failed: {e}")
                import traceback
                traceback.print_exc()
        
        # Aggressive cleanup
        del cameras, points, observations, image_names, graph, initial, result
        gc.collect()
        print(f"\n  Memory cleaned up")
        
        return True
        
    except Exception as e:
        print(f"ERROR: {type(e).__name__}: {e}")
        import traceback
        traceback.print_exc()
        
        # Save what we have
        save_results(all_results, RESULTS_FILE)
        
        # Force cleanup
        gc.collect()
        
        return False

# The plotting function only plots the first max_points landmark points to avoid computational overload.
def plot_dataset(idx, max_points=10000):
    """
    Plot reconstruction for a dataset that was run without plotting.
    This will re-run the optimization.
    
    Args:
        idx: dataset index
        max_points: maximum number of points to plot
    """
    global all_results
    
    name, bundle_file, list_file = DATASET_PATHS[idx]
    
    # Check if this dataset has been completed
    completed_names = [r['name'] for r in all_results]
    if name not in completed_names:
        print(f"Dataset {name} has not been run yet!")
        print(f"Run it first with: run_single_dataset({idx})")
        return
    
    print(f"\nPlotting reconstruction for: {name}")
    print("  WARNING: This will re-run the optimization")
    
    try:
        # Reload and optimize
        print("  Loading data...")
        cameras, points, observations, image_names = load_1dsfm_data(bundle_file, list_file)
        
        print("  Building graph...")
        graph, initial = build_factor_graph(cameras, points, observations)
        
        print("  Re-running optimization...")
        result, metrics = run_bundle_adjustment(
            graph, initial, 
            max_iterations=50,
            verbose=False
        )
        
        print("  Generating plot...")
        plot_reconstruction(
            cameras, points,
            initial, 
            result, 
            name,
            max_points=max_points,
            save_path=f"{name}_reconstruction.html"
        )
        
        # Cleanup
        del cameras, points, observations, image_names, graph, initial, result
        gc.collect()
        
        print(f"\n✓ Plot saved to: {name}_reconstruction.html")
        
    except Exception as e:
        print(f"ERROR: {e}")
        import traceback
        traceback.print_exc()

In [6]:
check_memory_status()
run_single_dataset(0, verbose=True, use_bundler_camera=False,
                   lambda_initial=100.0,      # Very conservative start
                   lambda_upper_bound=1e15)        # More iterations
force_cleanup()


Memory Status:
  Total: 31.1 GB
  Available: 4.0 GB
  Used: 26.0 GB (87.3%)
  Free: 2.8 GB

DATASET 1: alamo
  Loading 1DSfM file: /home/anatharv1/GTSAM-Project/BA_datasets/1dsfm/Alamo/colmap_bundle.out.bundle.out
  Parsing: 877 cameras, 99351 points
  Loaded 877 image names
  Loaded 877 cameras, 99351 points, 1116388 observations
Building factor graph...
  877 cameras, 99351 points, 1116388 observations
  Observation ranges: X=[-796.3, 797.1], Y=[-1052.6, 795.5]
  -> Observations are already centered
  Using Cal3_S2 camera model (no distortion)
  Valid cameras: 877/877
  Valid points: 99351/99351
  Using Huber robust loss function
  Added 1116388 projection factors
  Graph size: 1116390 factors
  Values size: 100228 variables
  Initial error: 1.58e+10
  NOTE: High initial error is expected for COLMAP Bundler exports
        Huber loss will handle outliers during optimization

LM Parameters:
  Max iterations: 50
  Initial lambda: 100.0
  Lambda factor: 10.0
  Lambda upper bound: 10000

✓ Plot saved to: alamo_reconstruction.html

  Memory cleaned up

Garbage collection completed

Memory Status:
  Total: 31.1 GB
  Available: 1.5 GB
  Used: 29.2 GB (95.3%)
  Free: 0.6 GB


In [7]:
check_memory_status()
run_single_dataset(1, verbose=True, use_bundler_camera=False,
                   lambda_initial=1e4,      # Very conservative start
                   lambda_upper_bound=1e15)        # More iterations
force_cleanup()


Memory Status:
  Total: 31.1 GB
  Available: 1.5 GB
  Used: 29.2 GB (95.3%)
  Free: 0.6 GB

DATASET 2: montreal_notre_dame
  Loading 1DSfM file: /home/anatharv1/GTSAM-Project/BA_datasets/1dsfm/Montreal_Notre_Dame/colmap_bundle.out.bundle.out
  Parsing: 571 cameras, 93987 points
  Loaded 571 image names
  Loaded 571 cameras, 93987 points, 887724 observations
Building factor graph...
  571 cameras, 93987 points, 887724 observations
  Observation ranges: X=[-797.1, 796.8], Y=[-795.8, 797.6]
  -> Observations are already centered
  Using Cal3_S2 camera model (no distortion)
  Valid cameras: 571/571
  Valid points: 93987/93987
  Using Huber robust loss function
  Added 887724 projection factors
  Graph size: 887726 factors
  Values size: 94558 variables
  Initial error: 2.53e+09
  NOTE: High initial error is expected for COLMAP Bundler exports
        Huber loss will handle outliers during optimization

LM Parameters:
  Max iterations: 50
  Initial lambda: 10000.0
  Lambda factor: 10.0
  L

✓ Plot saved to: montreal_notre_dame_reconstruction.html

  Memory cleaned up

Garbage collection completed

Memory Status:
  Total: 31.1 GB
  Available: 1.4 GB
  Used: 29.1 GB (95.4%)
  Free: 0.5 GB


In [8]:
check_memory_status()
run_single_dataset(2, verbose=True, use_bundler_camera=False,
                   lambda_initial=1e4,      # Very conservative start
                   lambda_upper_bound=1e15)        # More iterations
force_cleanup()


Memory Status:
  Total: 31.1 GB
  Available: 1.4 GB
  Used: 29.1 GB (95.5%)
  Free: 0.5 GB

DATASET 3: roman_forum
  Loading 1DSfM file: /home/anatharv1/GTSAM-Project/BA_datasets/1dsfm/Roman_Forum/colmap_bundle.out.bundle.out
  Parsing: 1490 cameras, 182050 points
  Loaded 1490 image names
  Loaded 1490 cameras, 182050 points, 1468509 observations
Building factor graph...
  1490 cameras, 182050 points, 1468509 observations
  Observation ranges: X=[-796.3, 796.9], Y=[-1042.5, 924.7]
  -> Observations are already centered
  Using Cal3_S2 camera model (no distortion)
  Valid cameras: 1490/1490
  Valid points: 182050/182050
  Using Huber robust loss function
  Added 1468509 projection factors
  Graph size: 1468511 factors
  Values size: 183540 variables
  Initial error: 5.88e+09
  NOTE: High initial error is expected for COLMAP Bundler exports
        Huber loss will handle outliers during optimization

LM Parameters:
  Max iterations: 50
  Initial lambda: 10000.0
  Lambda factor: 10.0
  

✓ Plot saved to: roman_forum_reconstruction.html

  Memory cleaned up

Garbage collection completed

Memory Status:
  Total: 31.1 GB
  Available: 1.6 GB
  Used: 28.9 GB (94.8%)
  Free: 0.6 GB


In [9]:
check_memory_status()
run_single_dataset(3, verbose=True, use_bundler_camera=False,
                   lambda_initial=1e4,      # Very conservative start
                   lambda_upper_bound=1e20)        # More iterations
force_cleanup()


Memory Status:
  Total: 31.1 GB
  Available: 1.6 GB
  Used: 28.9 GB (94.8%)
  Free: 0.6 GB

DATASET 4: picaddilly
  Loading 1DSfM file: /home/anatharv1/GTSAM-Project/BA_datasets/1dsfm/Piccadilly/colmap_bundle.out.bundle.out
  Parsing: 2994 cameras, 229448 points
  Loaded 2994 image names
  Loaded 2994 cameras, 229448 points, 2052807 observations
Building factor graph...
  2994 cameras, 229448 points, 2052807 observations
  Observation ranges: X=[-799.0, 797.7], Y=[-1133.3, 794.6]
  -> Observations are already centered
  Using Cal3_S2 camera model (no distortion)
  Valid cameras: 2994/2994
  Valid points: 229448/229448
  Using Huber robust loss function
  Added 2052807 projection factors
  Graph size: 2052809 factors
  Values size: 232442 variables
  Initial error: 3.94e+10
  NOTE: High initial error is expected for COLMAP Bundler exports
        Huber loss will handle outliers during optimization

LM Parameters:
  Max iterations: 50
  Initial lambda: 10000.0
  Lambda factor: 10.0
  La

✓ Plot saved to: picaddilly_reconstruction.html

  Memory cleaned up

Garbage collection completed

Memory Status:
  Total: 31.1 GB
  Available: 5.2 GB
  Used: 25.4 GB (83.3%)
  Free: 4.1 GB


In [10]:
check_memory_status()
run_single_dataset(4, verbose=True, use_bundler_camera=False,
                   lambda_initial=1e4,      # Very conservative start
                   lambda_upper_bound=1e20)        # More iterations
force_cleanup()


Memory Status:
  Total: 31.1 GB
  Available: 5.2 GB
  Used: 25.4 GB (83.2%)
  Free: 4.1 GB

DATASET 5: trafalgar
  Loading 1DSfM file: /home/anatharv1/GTSAM-Project/BA_datasets/1dsfm/Trafalgar/colmap_bundle.out.bundle.out
  Parsing: 2 cameras, 216 points
  Loaded 2 image names
  Loaded 2 cameras, 216 points, 432 observations
Building factor graph...
  2 cameras, 216 points, 432 observations
  Observation ranges: X=[-59.3, 715.9], Y=[-165.2, 464.3]
  -> Observations are already centered
  Using Cal3_S2 camera model (no distortion)
  Valid cameras: 2/2
  Valid points: 216/216
  Using Huber robust loss function
  Added 432 projection factors
  Graph size: 434 factors
  Values size: 218 variables
  Initial error: 3.57e+06

LM Parameters:
  Max iterations: 50
  Initial lambda: 10000.0
  Lambda factor: 10.0
  Lambda upper bound: 1e+20
  Lambda lower bound: 1e-10
  Relative error tol: 1e-05
  Absolute error tol: 1e-05

Running Bundle Adjustment...
  Initial error: 3.565883e+06
--------------

✓ Plot saved to: trafalgar_reconstruction.html

  Memory cleaned up

Garbage collection completed

Memory Status:
  Total: 31.1 GB
  Available: 5.2 GB
  Used: 25.5 GB (83.4%)
  Free: 4.0 GB


In [11]:
def generate_summary_report():
    """Generate comprehensive summary report with visualizations."""
    global all_results
    
    if not all_results:
        print("No results available! Run some datasets first.")
        return
    
    import pandas as pd
    import numpy as np
    try:
        import plotly.graph_objects as go
        from plotly.subplots import make_subplots
        use_plotly = True
    except ImportError:
        print("Plotly not available, summary table only")
        use_plotly = False
    
    # Create summary dataframe
    summary_df = pd.DataFrame([
        {
            'Dataset': r['name'],
            'Cameras': r['num_cameras'],
            'Points': r['num_points'],
            'Observations': r['num_observations'],
            'Time (s)': f"{r['time']:.1f}",
            'Max CPU (%)': f"{r['max_cpu']:.1f}",
            'Avg CPU (%)': f"{r['avg_cpu']:.1f}",
            'Max RAM (GB)': f"{r['max_ram_gb']:.2f}",
            'Avg RAM (GB)': f"{r['avg_ram_gb']:.2f}",
            'Initial Error': f"{r['initial_error']:.2e}",
            'Final Error': f"{r['final_error']:.2e}",
            'Iterations': r['iterations'],
            'Success': '✓' if r['success'] else '✗'
        }
        for r in all_results
    ])
    
    print("\n" + "="*100)
    print("SUMMARY OF ALL BUNDLE ADJUSTMENT RUNS")
    print("="*100)
    print(summary_df.to_string(index=False))
    print("="*100)
    
    if not use_plotly:
        return summary_df
    
    # Create interactive Plotly visualizations
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Optimization Time vs Problem Size',
            'Peak Memory Usage vs Problem Size',
            'CPU Utilization',
            'Optimization Error Reduction'
        ),
        specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
               [{'type': 'bar'}, {'type': 'bar'}]]
    )
    
    # Plot 1: Time vs Problem Size
    fig.add_trace(
        go.Scatter(
            x=[r['num_cameras'] for r in all_results],
            y=[r['time'] for r in all_results],
            mode='markers',
            marker=dict(size=12, color='steelblue', opacity=0.7),
            text=[r['name'] for r in all_results],
            hovertemplate='<b>%{text}</b><br>Cameras: %{x}<br>Time: %{y:.1f}s<extra></extra>',
            name='Time'
        ),
        row=1, col=1
    )
    fig.update_xaxes(title_text="Number of Cameras", row=1, col=1)
    fig.update_yaxes(title_text="Time (seconds)", row=1, col=1)
    
    # Plot 2: Max RAM vs Problem Size
    fig.add_trace(
        go.Scatter(
            x=[r['num_cameras'] for r in all_results],
            y=[r['max_ram_gb'] for r in all_results],
            mode='markers',
            marker=dict(size=12, color='coral', opacity=0.7),
            text=[r['name'] for r in all_results],
            hovertemplate='<b>%{text}</b><br>Cameras: %{x}<br>RAM: %{y:.2f} GB<extra></extra>',
            name='RAM'
        ),
        row=1, col=2
    )
    fig.update_xaxes(title_text="Number of Cameras", row=1, col=2)
    fig.update_yaxes(title_text="Max RAM (GB)", row=1, col=2)
    
    # Plot 3: CPU Usage
    names = [r['name'] for r in all_results]
    max_cpu = [r['max_cpu'] for r in all_results]
    avg_cpu = [r['avg_cpu'] for r in all_results]
    
    fig.add_trace(
        go.Bar(
            x=names,
            y=max_cpu,
            name='Max CPU',
            marker_color='steelblue',
            opacity=0.7,
            hovertemplate='<b>%{x}</b><br>Max CPU: %{y:.1f}%<extra></extra>'
        ),
        row=2, col=1
    )
    fig.add_trace(
        go.Bar(
            x=names,
            y=avg_cpu,
            name='Avg CPU',
            marker_color='lightblue',
            opacity=0.7,
            hovertemplate='<b>%{x}</b><br>Avg CPU: %{y:.1f}%<extra></extra>'
        ),
        row=2, col=1
    )
    fig.update_xaxes(title_text="Dataset", tickangle=45, row=2, col=1)
    fig.update_yaxes(title_text="CPU Usage (%)", row=2, col=1)
    
    # Plot 4: Error Reduction
    successful = [r for r in all_results if r['success']]
    if successful:
        error_reduction = [(1 - r['final_error']/r['initial_error'])*100 for r in successful]
        names_success = [r['name'] for r in successful]
        
        fig.add_trace(
            go.Bar(
                x=names_success,
                y=error_reduction,
                marker_color='lightgreen',
                opacity=0.7,
                hovertemplate='<b>%{x}</b><br>Error Reduction: %{y:.2f}%<extra></extra>',
                showlegend=False
            ),
            row=2, col=2
        )
        fig.update_xaxes(title_text="Dataset", tickangle=45, row=2, col=2)
        fig.update_yaxes(title_text="Error Reduction (%)", row=2, col=2)
    
    # Update layout
    fig.update_layout(
        title_text="GTSAM Bundle Adjustment - 1DSfM Scaling Analysis",
        height=800,
        showlegend=True,
        legend=dict(x=0.5, y=-0.15, xanchor='center', orientation='h')
    )
    
    # Show the plot
    fig.show()
    
    # Save as HTML
    fig.write_html("1dsfm_scaling_summary.html")
    print(f"\n✓ Interactive summary plot saved to: 1dsfm_scaling_summary.html")
    
    return summary_df


def print_comparison_table():
    """Print a detailed comparison table of all runs."""
    global all_results
    
    if not all_results:
        print("No results available!")
        return
    
    import pandas as pd
    
    # Detailed comparison
    comparison_df = pd.DataFrame([
        {
            'Dataset': r['name'],
            'Cameras': r['num_cameras'],
            'Points': r['num_points'],
            'Obs': r['num_observations'],
            'Time': r['time'],
            'Iters': r['iterations'],
            'Init Error': r['initial_error'],
            'Final Error': r['final_error'],
            'Reduction %': (1 - r['final_error']/r['initial_error'])*100,
            'Max RAM': r['max_ram_gb'],
            'Success': r['success']
        }
        for r in all_results
    ])
    
    print("\n" + "="*120)
    print("DETAILED COMPARISON TABLE")
    print("="*120)
    print(comparison_df.to_string(index=False))
    print("="*120)
    
    # Summary statistics
    print("\nSUMMARY STATISTICS:")
    print(f"  Total datasets completed: {len(all_results)}")
    print(f"  Total cameras: {comparison_df['Cameras'].sum():,}")
    print(f"  Total points: {comparison_df['Points'].sum():,}")
    print(f"  Total observations: {comparison_df['Obs'].sum():,}")
    print(f"  Total time: {comparison_df['Time'].sum():.1f} seconds ({comparison_df['Time'].sum()/60:.1f} minutes)")
    print(f"  Average time per dataset: {comparison_df['Time'].mean():.1f} seconds")
    print(f"  Max RAM usage: {comparison_df['Max RAM'].max():.2f} GB")
    print(f"  Average RAM usage: {comparison_df['Max RAM'].mean():.2f} GB")
    print(f"  Average error reduction: {comparison_df['Reduction %'].mean():.2f}%")
    print(f"  Success rate: {comparison_df['Success'].sum()}/{len(all_results)}")
    
    return comparison_df

In [12]:
summary_df = generate_summary_report()
comparison_df = print_comparison_table()


SUMMARY OF ALL BUNDLE ADJUSTMENT RUNS
            Dataset  Cameras  Points  Observations Time (s) Max CPU (%) Avg CPU (%) Max RAM (GB) Avg RAM (GB) Initial Error Final Error  Iterations Success
              alamo      877   99351       1116388    383.8         4.7         4.2         8.17         8.11      1.58e+10    8.05e+09          12       ✓
montreal_notre_dame      571   93987        887724     95.2         4.3         4.2         8.17         8.17      2.53e+09    2.50e+09           3       ✓
        roman_forum     1490  182050       1468509    306.6         4.2         4.0         8.19         8.19      5.88e+09    4.30e+09           7       ✓
         picaddilly     2994  229448       2052807   4329.0         4.1         3.9        15.75        15.75      3.94e+10    2.09e+10          19       ✓
          trafalgar        2     216           432      0.0         3.9         3.9        15.51        15.51      3.57e+06    3.57e+06           1       ✓



✓ Interactive summary plot saved to: 1dsfm_scaling_summary.html

DETAILED COMPARISON TABLE
            Dataset  Cameras  Points     Obs        Time  Iters   Init Error  Final Error  Reduction %   Max RAM  Success
              alamo      877   99351 1116388  383.785620     12 1.580984e+10 8.049081e+09    49.088166  8.173603     True
montreal_notre_dame      571   93987  887724   95.214493      3 2.525446e+09 2.497278e+09     1.115345  8.170624     True
        roman_forum     1490  182050 1468509  306.614156      7 5.879209e+09 4.296049e+09    26.928118  8.194389     True
         picaddilly     2994  229448 2052807 4328.956037     19 3.939976e+10 2.091905e+10    46.905632 15.750999     True
          trafalgar        2     216     432    0.003608      1 3.565883e+06 3.565883e+06     0.000000 15.509609     True

SUMMARY STATISTICS:
  Total datasets completed: 5
  Total cameras: 5,934
  Total points: 605,052
  Total observations: 5,525,860
  Total time: 5114.6 seconds (85.2 minutes)
  

### Observations & Inferences

**1. Alamo vs. Roman Forum**
* **Observation:** *Alamo* (877 cameras) took **383.8s** to optimize, whereas *Roman Forum* (1,490 cameras) finished in just **306.6s**, despite having nearly **double** the number of cameras.
* **Inference:** **Connectivity is a stronger predictor of runtime than dataset size.** The "Sparsity" column confirms this:
    * **Alamo:** 36.30% Sparsity (implies **63.7% Dense** connectivity). The high overlap between cameras creates a heavy, interconnected linear system.
    * **Roman Forum:** 74.07% Sparsity (implies **25.9% Dense** connectivity). The sparse structure allows the solver to exploit zero-blocks, making it faster even with $1.7\times$ more variables.

**2. Non-Linear Scaling (Piccadilly)**
* **Observation:** *Piccadilly* (2,994 cameras) took **4329s (~72 mins)**. While it has only $\sim2\times$ the cameras of *Roman Forum*, it took **$\sim14\times$ longer** to solve.
* **Inference:** The scaling is super-linear due to the **Graph Topology**. *Piccadilly* required **19 iterations** (vs. 7 for Roman Forum) to converge. Street-like, linear topologies often suffer from "gauge drift" or slow propagation of corrections across the chain, significantly increasing the iteration count required for convergence compared to compact scenes.

**3. Low Error Reduction compared to BAL**
* **Observation:** the 1dsfm datasets have less than 50% error reduction compared to BAL which averages 95% for the succesful ones.
* **Inference:** The COLMAP initialization for *Montreal* was already near-optimal (likely due to distinct, high-quality features on the cathedral facade). In contrast, *Alamo* started with a rougher initialization, requiring GTSAM to perform significant geometry correction (nearly halving the reprojection error).

**4. Failure Analysis (Trafalgar)**
* **Observation:** *Trafalgar* produced a degenerate graph with only **2 cameras** and **0.00s** optimization time.
* **Inference:** This confirms a **preprocessing failure**. The 15,000+ images likely overwhelmed the vocabulary tree matching step in COLMAP within the 2-hour constraint, failing to build a connected component. The bundle adjustment engine correctly identified there was nothing to optimize.

